In [1]:
!pip install pillow transformers torchvision
!pip install easyocr
!pip install crewai crewai-tools
!pip install numpy==1.26.4 --force-reinstall

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 193.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 24.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 31.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 166.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 143.5 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 92.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 23.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 137.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 66.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 140.5 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 56.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 

In [1]:
!pip uninstall transformers tokenizers -y

Found existing installation: transformers 4.57.3
Uninstalling transformers-4.57.3:
  Successfully uninstalled transformers-4.57.3
Found existing installation: tokenizers 0.22.1
Uninstalling tokenizers-0.22.1:
  Successfully uninstalled tokenizers-0.22.1


In [2]:
!pip install transformers==4.45.0 tokenizers==0.20.3

  Using cached tokenizers-0.20.3-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 183.4 MB/s eta 0:00:00
Using cached tokenizers-0.20.3-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [transformers] [transformers]


In [1]:
import os
import random
from PIL import Image, ImageDraw, ImageFont
import easyocr
import torch
import torchvision.transforms as transforms
from torchvision import models
from transformers import pipeline

In [43]:
#Synthetic claim form generator

def generate_synthetic_claim_form(output_path, claim_id):
    """
    Generates a synthetic insurance claim form image with sample details...
    """
    img = Image.new('RGB', (1200,1000), color=(255, 255, 255))
    d = ImageDraw.Draw(img)
    """
    try:
        font = ImageFont.truetype("arial.ttf", 20)
    except:
        font = ImageFont.load_default()
    """
    
    font = ImageFont.truetype("/usr/share/fonts/truetype/liberation2/LiberationSans-Regular.ttf", 20)
    
    details = [
        f"Claim ID: {claim_id}",
        f"Name: John Doe",
        f"Date: 2025-08-{random.randint(1,31)}",
        f"Policy No: P-{random.randint(100000, 999999)}",
        f"Incident: Vehicle accident at highway",
        f"Damage Estimate: ${random.randint(500,5000)}",
        f"Notes: Car collided with a vehicle at its front when it applied brakes suddenly, and the front bumper got damaged."
    ]

    y = 50
    for line in details:
        d.text((50, y), line, font=font, fill=(0, 0, 0))
        y +=40

    img.save(output_path)
    return output_path

In [23]:
#Text extraction using EasyOCR

ocr_reader = easyocr.Reader(['en'], download_enabled=False, gpu=False, model_storage_directory='/home/labuser/.EasyOCR/easyocr_models')

def extract_text_from_image(image_path):
    results = ocr_reader.readtext(image_path, detail=0)
    return "\n".join(results)

Using CPU. Note: This module is much faster with a GPU.


In [9]:
# Document Parser Agent

def document_parser_agent(extracted_text):
    """
    Parses claim text into structured data.
    """
    data = {}
    for line in extracted_text.split("\n"):
        if ":" in line:
            key, value = line.split(":", 1)
            data[key.strip()] = value.strip()
    return data

In [56]:
# Fraud Detection Agent

def fraud_detection_agent(parsed_data):
    """
    Fraud check - Flags high damage claims.
    """
    damage_value = 0
    if "Damage Estimate" in parsed_data:
        try:
            damage_value = int(parsed_data["Damage Estimate"].replace("$", "").strip())
        except:
            pass
    if damage_value > 4000:
        return "Potential Fraud: High damage estimate"
    return "No fraud indicators found."


resnet = models.resnet18(pretrained=True)
resnet.eval()

transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
])


/home/labuser/anaconda3/lib/python3.13/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/labuser/anaconda3/lib/python3.13/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [57]:
# Accident Image Analyzer Agent

def accident_image_analyzer(image_path):
    """
    Analyzes accident image and classifies severity.
    """
    try:
        img = Image.open(image_path).convert("RGB")
        tensor = transform(img).unsqueeze(0)
        with torch.no_grad():
            features = resnet(tensor)
        score = torch.mean(features).item()
        if score < 0:
            severity = "Low"
        elif score < 5:
            severity = "Medium"
        else:
            severity = "High"
        return {"severity": severity, "score": round(score,2)}
    except Exception as e:
        return {"severity": "Unknown", "error": str(e)}


In [58]:
# Risk Score Agent

def risk_scorer_agent(fraud_result, image_analysis):
    """
    Assigns a risk score from 0–100 based on fraud + image severity.
    """
    base_score = 20
    if "Potential Fraud" in fraud_result:
        base_score += 40
    if image_analysis.get("severity") == "High":
        base_score += 30
    elif image_analysis.get("severity") == "Medium":
        base_score += 15
    return min(100, base_score)


In [82]:
# Summarizer Agent

def summarizer_agent(parsed_data):
    """
    Summarizes the claim.
    """
    name = parsed_data.get("Name", "Unknown")
    incident = parsed_data.get("Incident", "No details")
    notes = parsed_data.get("Notes", "No details")
    return f"{name} reported '{incident}'. {notes}."


summarizer = pipeline("summarization", model = "facebook/bart-large-cnn")

def llm_summarize(text):
    return summarizer(text, max_length=15, min_length=10, do_sample=False)[0]['summary_text']


/home/labuser/anaconda3/lib/python3.13/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [60]:
#Orchestrator

def run_claim_investigation():
    claim_image_path = "/home/labuser/Desktop/Project/synthetic_claim.jpg"

    generate_synthetic_claim_form(claim_image_path, claim_id=random.randint(1000,9999))
    print(f"Synthetic claim form generated: {claim_image_path}")

    extracted_text = extract_text_from_image(claim_image_path)
    print("\nExtracted Text:\n", extracted_text)

    parsed_data = document_parser_agent(extracted_text)
    print("\nParsed Data:\n", parsed_data)

    fraud_result = fraud_detection_agent(parsed_data)
    print("\nFraud Detection Result:\n", fraud_result)

    image_analysis = accident_image_analyzer(claim_image_path)
    print("\nImage Analysis Result:\n", image_analysis)

    risk_score = risk_scorer_agent(fraud_result, image_analysis)
    print("\nRisk Score:", risk_score)

    claim_summary = summarizer_agent(parsed_data)
    print("\nClaim Summary:\n", claim_summary)

    improved_summary = llm_summarize(claim_summary)
    print("\nLLM Enhanced Summary:\n", improved_summary)

In [83]:
run_claim_investigation()

Synthetic claim form generated: /home/labuser/Desktop/Project/synthetic_claim.jpg


/home/labuser/anaconda3/lib/python3.13/site-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)



Extracted Text:
 Claim ID: 6524
Name: John Doe
Date: 2025-08-23
Policy No: P-561377
Incident: Vehicle accident at highway
Damage Estimate: $4295
Notes: Car collided with a vehicle at its front when it applied brakes suddenly, and the front bumper got damaged

Parsed Data:
 {'Claim ID': '6524', 'Name': 'John Doe', 'Date': '2025-08-23', 'Policy No': 'P-561377', 'Incident': 'Vehicle accident at highway', 'Damage Estimate': '$4295', 'Notes': 'Car collided with a vehicle at its front when it applied brakes suddenly, and the front bumper got damaged'}

Fraud Detection Result:
 Potential Fraud: High damage estimate

Image Analysis Result:
 {'severity': 'Medium', 'score': 0.0}

Risk Score: 75

Claim Summary:
 John Doe reported 'Vehicle accident at highway'. Car collided with a vehicle at its front when it applied brakes suddenly, and the front bumper got damaged.

LLM Enhanced Summary:
 Car collided with a vehicle at its front when it applied brakes


In [86]:
#CrewAI Integration

from crewai import Agent, Task, Crew, Process
from crewai.tools import tool
import json


@tool("parse_document")
def parse_document_tool(text: str) -> str:
    """Parses raw text into a structured JSON format using the document parser agent."""
    return json.dumps(document_parser_agent(text))

@tool("fraud_check")
def fraud_check_tool(parsed_json: str) -> str:
    """Analyzes a parsed JSON document to identify potential fraudulent patterns or inconsistencies."""
    parsed = json.loads(parsed_json)
    return json.dumps(fraud_detection_agent(parsed))

@tool("analyze_image")
def analyze_image_tool(image_path: str) -> str:
    """Analyzes an image for inconsistencies."""
    return json.dumps(accident_image_analyzer(image_path))

@tool("score_risk")
def score_risk_tool(fraud_json: str, image_json: str) -> str:
    """Assigns a score risk."""
    fraud = json.loads(fraud_json)
    image = json.loads(image_json)
    return json.dumps({"risk_score": risk_scorer_agent(fraud, image)})

@tool("summarize_claim")
def summarize_claim_tool(parsed_json: str, fraud_json: str, image_json: str, risk_json: str) -> str:
    """Summarizes the claim."""
    parsed = json.loads(parsed_json)
    fraud = json.loads(fraud_json)
    image = json.loads(image_json)
    risk = json.loads(risk_json).get("risk_score", 0)
    return llm_summarize(summarizer_agent(parsed))

In [100]:
# Define CrewAI agents

os.environ["OPENAI_API_KEY"] = "sk-7Y_G5KTkC-7sG_sHbFWrGw"

doc_agent = Agent(
    role="Document Parser",
    goal="Extract claim fields accurately from documents.",
    backstory="An expert AI specialized in parsing complex insurance documents and extracting relevant data fields with high accuracy.",
    tools=[parse_document_tool],
    verbose=True,
    allow_delegation=False
)

fraud_agent = Agent(
    role="Fraud Detector",
    goal="Detect suspicious claims and flag potential fraud cases.",
    backstory="A seasoned fraud analysis AI with years of experience in identifying patterns and anomalies indicative of insurance fraud.",
    tools=[fraud_check_tool],
    verbose=True,
    allow_delegation=True
)

image_agent = Agent(
    role="Accident Image Analyzer",
    goal="Assess accident severity and damage based on provided images.",
    backstory="A computer vision specialist AI capable of analyzing accident imagery to determine the extent of damage and severity.",
    tools=[analyze_image_tool],
    verbose=True,
    allow_delegation=False
)

risk_agent = Agent(
    role="Risk Scorer", 
    goal="Assign numerical risk score based on claim complexity and history", 
    backstory=(
        "You are a Senior Actuary with 15 years of experience in insurance risk modeling. "
        "Your expertise lies in quantifying uncertainty and identifying high-stakes claims "
        "that require manual intervention."
    ),
    tools=[score_risk_tool],
    verbose=True
)


summary_agent = Agent(
    role="Summarizer", 
    goal="Write a concise, professional claim summary for final approval", 
    backstory=(
        "You are a Claims Communication Specialist. You excel at synthesizing complex "
        "investigative data into clear, actionable executive summaries that help "
        "decision-makers approve or deny claims quickly."
    ),
    tools=[summarize_claim_tool],
    verbose=True
)




crew = Crew(
    agents=[doc_agent, fraud_agent, image_agent, risk_agent, summary_agent],
    process=Process.sequential
)

In [102]:
# Run CrewAI orchestrated pipeline
result = crew.run()
print(result)

AttributeError: 'Crew' object has no attribute 'run'

In [103]:
#https://huggingface.co/xiaoyao9184/easyocr/blob/master/english_g2.pth

#https://huggingface.co/xiaoyao9184/easyocr/blob/master/craft_mlt_25k.pth